In [27]:
import pandas as pd
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


data = pd.read_csv("tweets.csv", sep = ',')

[nltk_data] Downloading package punkt to /home/nomdebrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nomdebrew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
data

,user_id,user_key,created_at,created_str,retweet_count,retweeted,favorite_count,text,tweet_id,source,hashtags,expanded_urls,posted,mentions,retweeted_status_id,in_reply_to_status_id
0,2.532612e+09,kathiemrr,1.488207e+12,2017-02-27 14:54:00,NaN,NaN,NaN,#ThingsDoneByMistake kissing auntie in the lips,8.362279e+17,NaN,"[""ThingsDoneByMistake""]",[],POSTED,[],NaN,NaN
1,2.531160e+09,traceyhappymom,1.471273e+12,2016-08-15 14:50:20,NaN,NaN,NaN,RT @mc_derpin: #TheOlderWeGet the more pessimi...,7.651989e+17,NaN,"[""TheOlderWeGet""]",[],POSTED,[],NaN,NaN
2,NaN,evewebster373,1.435701e+12,2015-06-30 21:56:09,NaN,NaN,NaN,RT @dmataconis: Ready To Feel Like A Failure? ...,6.160023e+17,NaN,[],[],POSTED,[],NaN,NaN
3,4.840552e+09,blacktolive,1.474013e+12,2016-09-16 08:04:48,18.0,False,17.0,Amen! #blacklivesmatter https://t.co/wGffaOqgzl,7.766933e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""Blacklivesmatter""]",[],POSTED,[],NaN,NaN
4,1.694026e+09,jacquelinisbest,1.474228e+12,2016-09-18 19:46:25,0.0,False,0.0,RT @NahBabyNah: Twitchy: Chuck Todd caught out...,7.775946e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","[""WakeUpAmerica""]","[""http://ln.is/twitchy.com/loriz-31/3yafU""]",POSTED,"[""nahbabynah""]",7.775915e+17,NaN
5,2.587101e+09,judelambertusa,1.460029e+12,2016-04-07 11:37:45,NaN,NaN,NaN,RT @mcicero10: #BernieSanders #Trump people sh...,7.180401e+17,NaN,"[""BernieSanders""]",[],POSTED,[],NaN,NaN
6,1.647046e+09,carriethornthon,1.476133e+12,2016-10-10 21:04:06,0.0,False,0.0,RT @ItsJustJaynie: @HillaryClinton The undecid...,7.855867e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],[],POSTED,"[""itsjustjaynie"",""hillaryclinton"",""cnn""]",7.853264e+17,NaN
7,1.657754e+09,johnbranchh,1.447286e+12,2015-11-11 23:54:42,NaN,NaN,NaN,@TodayCleveland 'no way',6.645921e+17,NaN,[],[],POSTED,[],NaN,NaN
8,7.374200e+17,march_for_trump,1.475376e+12,2016-10-02 02:35:35,1.0,False,0.0,"@NickTomaWBRE Hi, Nick! We're holding a ""Miner...",7.824087e+17,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],"[""https://twitter.com/i/web/status/78240866138...",POSTED,"[""nicktomawbre""]",NaN,NaN
9,2.620870e+09,puredavie,1.482331e+12,2016-12-21 14:35:32,NaN,NaN,NaN,What. Is. A. Resolution #My4WordNewYearsResolu...,8.115809e+17,NaN,"[""My4WordNewYearsResolution""]",[],POSTED,[],NaN,NaN


In [22]:
data.shape

(203451, 16)

In [23]:
textData = data['text']

In [24]:
textData

0           #ThingsDoneByMistake kissing auntie in the lips
1         RT @mc_derpin: #TheOlderWeGet the more pessimi...
2         RT @dmataconis: Ready To Feel Like A Failure? ...
3           Amen! #blacklivesmatter https://t.co/wGffaOqgzl
4         RT @NahBabyNah: Twitchy: Chuck Todd caught out...
5         RT @mcicero10: #BernieSanders #Trump people sh...
6         RT @ItsJustJaynie: @HillaryClinton The undecid...
7                                  @TodayCleveland 'no way'
8         @NickTomaWBRE Hi, Nick! We're holding a "Miner...
9         What. Is. A. Resolution #My4WordNewYearsResolu...
10        Lifetime movie   your pet   psycho neighbor = ...
11        RT @Conservatexian: New post: "UN alarm that m...
12        RT @HillaryClinton: This one's for you, Hillar...
13        RT @leonpui_: Hillary Clinton, Obama and the D...
14        5th grade. When the second plane hit, I though...
15        #teapartynews #teaparty #theteaparty #politics...
16        #TrumpBecause #DonaldTrump wil

In [35]:
for i in range(len(textData)):
    print(word_tokenize(textData[i]))


['#', 'ThingsDoneByMistake', 'kissing', 'auntie', 'in', 'the', 'lips']
['RT', '@', 'mc_derpin', ':', '#', 'TheOlderWeGet', 'the', 'more', 'pessimistic', 'we', 'are', 'https', ':', '//t.co/zS3jHZJl8P']
['RT', '@', 'dmataconis', ':', 'Ready', 'To', 'Feel', 'Like', 'A', 'Failure', '?', 'Joan', 'Of', 'Arc', 'Was', 'Only', '19', 'When', 'She', 'Was', 'Burned', 'At', 'The', 'Stake', 'http', ':', '//t.co/S2j1IFm4y9']
['Amen', '!', '#', 'blacklivesmatter', 'https', ':', '//t.co/wGffaOqgzl']
['RT', '@', 'NahBabyNah', ':', 'Twitchy', ':', 'Chuck', 'Todd', 'caught', 'out', 'there', 'shilling', 'for', 'Hillary', 'Clinton', 'The', 'post', 'BUSTED', ':', 'Adam', 'Baldwi', '...', 'https', ':', '//t.co/ay28pMpDw6', '#', '…']
['RT', '@', 'mcicero10', ':', '#', 'BernieSanders', '#', 'Trump', 'people', 'should', 'rally', 'TOGETHER', 'against', 'the', 'establishment', 'who', 'is', '💩-ing', 'on', 'both', 'choices', '#', 'thefix']
['RT', '@', 'ItsJustJaynie', ':', '@', 'HillaryClinton', 'The', 'undecided', 

['Trump', 'campaign', 'CEO', 'did', "n't", 'want', 'daughters', "'going", 'to', 'school', 'with', 'Jews', ',', "'", 'ex-wife', 'says', '#', 'politics']
['RT', '@', 'kiddstclair', ':', 'I', 'save', 'all', 'my', 'change', 'throughout', 'the', 'year', '&', 'amp', ';', 'throw', 'it', 'at', 'strippers', 'for', 'the', 'holidays', '#', 'SecondhandGifts']
['RT', '@', 'ArcticFox2016', ':', '★★★', 'Patriots', 'Who', 'Dare', '...', 'Join', 'our', 'fight', 'to', 'save', 'America', '!', '➠', 'Click', 'Here', 'https', ':', '//t.co/TM3sg8AXwm', '#', 'BB4SP', 'https', ':', '//t.co/7Q4b4E…']
['RT', '@', 'cyvault', ':', 'MSNBC', 'panel', 'SHOCKED', 'to', 'learn', '#', 'Trump', 'warned', 'about', '9/11', 'in', 'Jan', '2000', ',', '19+', 'mo', 'before', 'attack', '!', '#', 'SundayMorning', '#', 'NeverForget', 'https', ':', '//t…']
['RT', '@', 'DanteB4u', ':', 'WATCH', ':', 'How', '#', 'GOP', '&', 'amp', ';', '#', 'ALEC', '(', 'American', 'Legislative', 'Exchange', 'Council', ')', 'Circumvent', 'Democracy'

['RT', '@', 'LorettaBower3', ':', 'The', 'GOP', 'is', 'actively', 'attacking', 'the', 'ACA', '.', 'Their', 'plan', ',', 'AHCA', 'will', 'never', 'pass', ',', 'so', 'end', 'result', 'is', 'no', 'healthcare', 'plan', '.', 'That', 'is…']
['RT', '@', 'RealJamesWoods', ':', 'Hillary', 'Clinton', 'Private', 'Email', 'Created', 'Internal', 'Rift', '-', 'A', 'look', 'at', 'the', 'rats', 'nibbling', 'at', 'each', 'other', "'s", 'gonads', '.', '#', 'Crooked', 'htt…']
['RT', '@', 'adriennefunny', ':', '@', 'tonyposnanski', 'Ha', 'ha', 'ha', '...', '.', '.Hello', 'Tony', '!', '!', '!', 'Happy', 'Holidays', '!']
['RT', '@', 'nine_oh', ':', '#', 'Listen2', 'Castle', 'On', 'The', 'Hill', 'by', '@', 'edsheeran', 'on', 'app', 'at', 'https', ':', '//t.co/iChLzWqg0Y']
['RT', '@', 'El_Timbo_Libre', ':', '@', 'DonCheadle', 'Just', 'watched', 'Miles', 'Ahead', '.', 'Speechless', '.']
['Pew', 'Song', '#', 'MakeMusicReligious']
['RT', '@', 'prd_2b_USA', ':', 'I', 'wish', 'there', 'was', 'just', 'as', 'much', 

['RT', '@', 'asamjulian', ':', '@', 'HillaryClinton', 'Which', 'means', 'you', 'have', 'no', 'place', 'in', 'America', '!']
['RT', '@', 'tpartynews', ':', 'Mike', 'Huckabee', ':', '``', 'Hillary', 'Clinton', 'is', 'an', 'elitist', 'snob', '.', 'She', 'believes', 'she', "'s", 'smarter', 'than', 'you', '.', "''", '#', 'BasementDwellers', 'https', ':', '//t.co/kd…']
['RT', '@', 'Conservatexian', ':', 'George', 'Carlin', '-', 'Political', 'Correctness', 'is', 'fascism', 'pretending', 'to', 'be', 'Manners', '...', '...', '...', '...', '...', '...', 'https', ':', '//t.co/z2dwSioHOS']
['RT', '@', 'annietfirst', ':', '@', 'LorettaLynch', '@', 'SenWarren', '@', 'BarackObama', '@', 'POTUS', '@', 'HillaryClinton', 'We', 'will', '.', "'", "''", 'NEVER', 'FORGET', 'BENGHAZI', "''", "''LOOK", 'AT', 'THIS', 'PICT…']
['I', 'vape', '#', 'MakeMeHateYouInOnePhrase']
['#', 'WearHoodieForTrayvon', 'and', 'stand', 'up', 'for', 'your', 'rights', '!', '#', 'WeAreAllTrayvon']
['SMDH😢', '#', 'SanBernardino', 'c

['RT', '@', 'tiffjhuang', ':', 'Last', 'night', ',', 'I', 'sent', 'this', 'to', 'De', 'Blasio', ',', 'the', '#', 'NYC', 'Comptroller', ',', 'and', 'my', 'city', 'councilperson', '.', 'Please', 'do', 'the', 'same', '#', 'NoDAPL', '#', 'NextToDi…']
['RT', '@', 'martinez4_david', ':', '@', 'FoxNews', '@', 'BarackObama', 'No', 'progress', 'considering', 'you', 'have', 'been', 'screwing', 'our', 'country', 'for', '8', 'years', '.', 'Do', "n't", 'try', 'to', 'sugar', 'coate…']
['Brunell', 'Kyei', 'Stands', 'Strong', 'w', 'Trump', '—Tells', 'it', 'like', 'it', 'is', '#', 'AmericaFirst', '#', 'voteTrumpPence', 'https', ':', '//t.co/V7vEe9Xgsc', '#', 'TrumpForPresident']
['Gute', '#', 'Nacht', ',', 'meine', 'Lieben', '!', 'Träumt', 'süß', '!', 'https', ':', '//t.co/oHQSnt4MlW']
['RT', '@', 'ChristiChat', ':', 'MT', '@', 'FreedomChild3', ':', 'You', 'can', 'not', 'fool', 'all', 'of', 'us', '!', '#', 'ClimateChange', '#', 'WakeUpAmerica', '#', 'tcot', 'http', ':', '//t.co/vW6xqdSxTO', '#', 'PJNET

['RT', '@', 'PcolaBucsfan', ':', 'KANYE', 'WEST', 'SUPPORTS', 'TRUMP', 'https', ':', '//t.co/cZhELHl78l', 'via', '@', 'YouTube']
["'The", 'movement', 'of', 'Muslims', ',', 'into', 'Europe', 'and', 'the', 'US', 'is', 'a', 'planned', 'invasion..', "'", '~', 'Michele', 'Bachmann', 'https', ':', '//t.co/pGGsIgO4kY']
['RT', '@', 'Tenthltr2u', ':', '“', 'Scientific', 'racism', '”', 'is', 'on', 'the', 'rise', 'on', 'the', 'right', '.', 'But', 'it', '’', 's', 'been', 'lurking', 'there', 'for', 'years', '.', 'https', ':', '//t.co/fdMjmDZP6B', 'https', ':', '//t.c…']
['RT', '@', 'AACONS', ':', 'How', 'can', 'someone', 'so', 'hostile', 'to', 'African', 'Americans', 'demand', 'such', 'loyalty', 'from', 'them', '?', '#', 'dk', 'https', ':', '//t.co/Q4PASrXwqv']
['RT', '@', 'a_malahov', ':', 'Thanks', 'to', '@', 'realDonaldTrump', 'and', '@', 'HillaryClinton', 'for', 'interesting', 'jogging', 'session', ')', 'https', ':', '//t.co/qF4YNwN0Ms']
['Was', 'once', 'suicided', 'by', 'the', 'Clinton', "'s",

['RT', '@', 'nobamanoway', ':', 'It', "'s", 'ok', ',', 'Julie', 'says', 'we', "'re", 'bigots', 'if', 'we', 'do', "n't", 'let', 'them', '.', '#', 'Outnumbered', '@', 'TheFive', 'Why', 'are', 'kids', 'allowed', 'to', 'read', 'the', 'Quran', '?', 'https…']
['RT', '@', 'DorothieBell', ':', 'This', 'is', 'getting', 'serious', '.', 'This', 'calls', 'for', 'serious', 'action', '!', 'Sign', 'this', 'petition', 'now', '!', 'https', ':', '//t.co/Rgfd9QgKeQ', 'https', ':', '//t.co/RLfH…']
['RT', '@', 'jemelehill', ':', 'So', 'black', 'women', 'voted', 'for', 'the', 'greater', 'promise', 'of', 'gender', 'equality', 'and', 'this', 'group', 'chose', 'privilege', '.', 'https', ':', '//t.co/Wv0cHI5fpR']
['#', 'ThingsYouCantIgnore', 'Alcoholism']
['RT', '@', 'preciousliberty', ':', 'Here', 'is', '#', 'Trump', 'opposing', 'the', 'Iraq', 'War', 'in', 'early', '2003', 'https', ':', '//t.co/M3O7TpOLXL', '#', 'DemExit', '#', 'FeelTheBern', '#', 'Blacks4Trump', '#', 'Indep…']
['#', 'toavoidworki', 'remember'

['RT', '@', 'Teejandyande1', ':', 'Nor', 'go', 'Mess', 'up', 'with', 'the', '#', 'TagTeam', '@', 'teejandyande1', '@', 'Cruzyofficial', '#', 'DropIt', 'This', 'song', '@', 'billboard', 'go', 'celebrate', 'am', 'kpokpo', '.', 'Lm…']
['RT', '@', 'AndrewHashtagW', ':', 'I', '’', 'll', 'give', 'you', 'all', 'what', 'you', 'want', '.', 'If', 'you', 'want', 'only', 'a', 'pen', 'and', 'a', 'notebook', '#', 'MyCampaignSlogan', 'http', ':', '//t.co/ldmL7YN6GU']
['RT', '@', 'BoogsMalone', ':', 'GET', 'THE', 'HOOK', 'UP', 'ON', 'A', 'DOPE', 'BOY', 'BOOGS', 'HOOK', 'https', ':', '//t.co/7fSzjERZ0q', '#', 'Hiphopbeats']
['RT', '@', 'JudicialWatch', ':', 'Judicial', 'Watch', 'keeps', 'an', 'eye', 'on', 'BOTH', 'political', 'parties', 'and', 'all', 'branches', 'of', 'government', '.', 'We', 'work', 'for', 'you', '!', 'https', ':', '//t.co/Hwgp…']
['RT', '@', 'sardnas51', ':', 'Clinton', 'Benefits', 'After', 'Foul', 'Play', 'Suspected', 'in', 'Keeping', 'Green', 'Party', 'Off', 'Nevada', 'Ballot', 'ht

TypeError: expected string or bytes-like object

In [28]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

In [29]:
preprocessSentence(textData[2])

NameError: name 'preprocessSentence' is not defined